In [ ]:
from sklearn.model_selection import KFold

In [ ]:
## 10 Fold Validation
kfold = KFold(10,
                 shuffle = True,
                 random_state = 732)

In [ ]:
# Making a KFold splittable dataframe that ignores Home and Away 
# and has a unique ID column for index

valid_train_df = train_games_df
valid_train_df.drop(["id"], axis=1, inplace=True)
valid_train_df["id"] = valid_train_df.index

In [ ]:
mses

In [ ]:
pd.options.mode.chained_assignment = None  # default='warn'

## make an array of zeros that will hold our mses
# Compare lasso to elastic net to just the mean (mean should perform worst I hope)
mses = np.zeros((3, 10))

## This keeps track of what split we are on
i = 0

## fill in what is missing in the for loop declaration
for train_index, test_index in kfold.split(valid_train_df):
    
    ### Training
    fb_t_t = valid_train_df.iloc[train_index]
    
    ### Holdout set
    fb_ho = valid_train_df.iloc[test_index]

# Creating a new train thingy for kfold that includes points for validation
    X_train = []
    y_train = []
    X_test = []
    y_test = []

    kfold_train_ids = set(fb_t_t["id"]) 
    kfold_ho_ids = set(fb_ho["id"]) 

    for id in kfold_train_ids:
        temp = valid_train_df.loc[valid_train_df["id"] == id]

        y_train.append(np.array(temp["points"]))
        temp.drop(["side", "points", "id"], axis=1, inplace=True)

        temp = temp.to_numpy()

        X_train.append(temp)
    
    X_train = np.array(X_train)
    y_train = np.array(y_train)


    X_train = X_train.reshape(-1, 29)


    for id in kfold_ho_ids:
        temp_test = valid_train_df.loc[valid_train_df["id"] == id]

        y_test.append(np.array(temp_test["points"]))
        temp_test.drop(["side", "points", "id"], axis=1, inplace=True)

        temp_test = temp_test.to_numpy()

        X_test.append(temp)

    X_test = np.array(X_test)
    y_test = np.array(y_test)
    X_test = np.array(X_test).reshape(-1, 29)


    # Re-scaling the data before each model test
    scaler = StandardScaler()
    scaler.fit(X_train)
    X_train_scaled = scaler.transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    ### This is Model 0 ###
    ## take the mean W from the training set
    ## we need predictions for the entire holdout set
    pred_M = fb_t_t.points.mean() * np.ones(len(X_test))
    
    ### This is Model 1 ###
    ## Lasso regression, alpha = 1
    ## Define the LinearRegression object
    model_L = Lasso(alpha=1)
    
    ## fit model_L on the training data, fb_t_t
    model_L.fit(X_train_scaled, y_train)
    
    ## get the prediction on holdout set
    pred_L = model_L.predict(X_test_scaled)
    
    # Model 2, Elastic Net
    model_E = ElasticNet(alpha=1)
    
    # fit model_E on the training data, fb_t_t
    model_E.fit(X_train_scaled, y_train)
    
    # ## get the prediction on holdout set
    pred_E = model_E.predict(X_test_scaled)


    ### Recording the MSES ###
    ## mean_absolute error takes in the true values, then the predicted values
    ## model 0
    mses[0,i] = mean_absolute_error(y_test, pred_M)
    
    ## model 1
    mses[1,i] = mean_absolute_error(y_test, pred_L)
    
    ## model 2
    mses[2,i] = mean_absolute_error(y_test, pred_E)
    
    # increase i by 1
    i = i + 1
    print('KFold '+ str(i) + ' complete')

In [ ]:
## This figure will compare the performance
plt.figure(figsize=(8,10))

plt.scatter(np.zeros(10), 
            mses[0,:], 
            s=60, 
            c='white',
            edgecolor='black',
            label="Single Split")
plt.scatter(np.ones(10), 
            mses[1,:], 
            s=60, 
            c='white',
            edgecolor='black')
plt.scatter(2*np.ones(10), 
            mses[2,:], 
            s=60, 
            c='white',
            edgecolor='black')

plt.scatter([0,1,2], 
            np.mean(mses, axis=1), 
            s=60, 
            c='r',
            marker='X',
            label="Mean")

plt.legend(fontsize=12)

plt.xticks([0,1,2],["Baseline", "Model 1", "Model 2"], fontsize=10)
plt.yticks(fontsize=10)

plt.xlabel("Model", fontsize=12)
plt.ylabel("MSE", fontsize=12)

plt.show()